[View in Colaboratory](https://colab.research.google.com/github/assaflehr/language-style-transfer/blob/master/notebooks/drnet_nlp_dataset.ipynb)

In [1]:
#!pip install spacy   #can take few minutes
#!python -m spacy download en
!pip install git+https://github.com/fastai/fastai.git
!pip install torch -U # 0.4 at-least

  Cloning https://github.com/fastai/fastai.git to /tmp/pip-req-build-bojptk6m
    100% |████████████████████████████████| 1.5MB 7.9MB/s 
    100% |████████████████████████████████| 112kB 10.7MB/s 
    100% |████████████████████████████████| 184kB 10.1MB/s 
    100% |████████████████████████████████| 3.4MB 8.5MB/s 


    100% |████████████████████████████████| 496.4MB 29kB/s 
    100% |████████████████████████████████| 51kB 12.9MB/s 
    100% |████████████████████████████████| 61kB 13.2MB/s 
    100% |████████████████████████████████| 51kB 22.1MB/s 
    100% |████████████████████████████████| 2.2MB 6.1MB/s 
    100% |████████████████████████████████| 13.7MB 2.0MB/s 
    100% |████████████████████████████████| 92kB 26.8MB/s 


    100% |████████████████████████████████| 112kB 25.2MB/s 
    100% |████████████████████████████████| 71kB 27.4MB/s 
    100% |████████████████████████████████| 81kB 25.5MB/s 
  Running setup.py bdist_wheel for fastai ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-sqgcnxmq/wheels/cf/46/39/b2d08762125ed2376861976ab2c4ac30c029b86e375735d9b8
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | /

 - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
  Running setup.py bdist_wheel for feather-format ... - done
  Stored in directory: /content/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988a9c1
  Running setup.py bdist_wheel for pandas-summary ... - done
  Stored in directory: /content/.cache/pip/wheels/86/a9/6b/36b727a9ec687dac08bd3d0501cddd7b1a223943513eb04a03
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built fastai bcolz feather-format pandas-summary torchtext
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
torchvision 0.2.1 has requirement pillow>=4.1.1, bu

    100% |████████████████████████████████| 484.0MB 13kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5bf1a000 @  0x7f5ad433b1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
fastai 0.7.0 has requirement torch<0.4, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 0.3.1
    Uninstalling torch-0.3.1:
      Successfully uninstalled torch-0.3.1


In [2]:
import torch.nn as nn
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *
from fastai.core import *
import fastai
from torch.utils.data import Dataset, DataLoader
import numpy as np
import time
from fastai.imports import *
from fastai.torch_imports import *
from fastai.io import *
%matplotlib inline

class TimeDataset(BaseDataset):
  
  def __init__(self,length=1e3): #1e9 means 1970-2001
    #super().__init__()
    self.length = length
    self.formats = ["%b %d %Y %H:%M:%S","%d %b %Y %H:%M:%S", "%m/%b/%Y %H:%M:%S"]
    self.x_dim = (10,)
    self.y_dim = (len(self.formats),)
    
  def __len__(self):
    return int(self.length)
  
  def transform(self,x):
    return np.array([ord(c) for c in list(x)][:self.x_dim[0]],np.float32)/100 #chars are up to that

  def __getitem__(self,idx):
    """ choose one random format and format time idx with it"""
    
    # random seed is initialized before each batch, we use idx to be different
    fs = len(self.formats)
    
    #idx = 666
    #formats = np.remainder(idx + np.random.randint(0,fs,size=2),fs)
    f=np.random.randint(0,fs)
    
    x = time.strftime(self.formats[f],time.gmtime(idx))
    #print (x)
    x= self.transform(x)
    
    y =  np.array(f ,np.int64 ) #0,1,2 must be Long to enalbe using NNLLoss
    #y = np.eye(fs, dtype='int64')[f]          #y as one-hot (wastefull a bit)
    
    
    return  (x, y)

dataset = TimeDataset(1e3)          
torch_dataloader = DataLoader(TimeDataset(1e3), batch_size=3, shuffle=False, num_workers=2) 
for _ in range (2):
  x,y = next(iter(torch_dataloader))
  print (x.shape,y.shape)
  print (x,y)





torch.Size([3, 10]) torch.Size([3])
tensor([[ 0.4800,  0.4900,  0.3200,  0.7400,  0.9700,  1.1000,  0.3200,
          0.4900,  0.5700,  0.5500],
        [ 0.7400,  0.9700,  1.1000,  0.3200,  0.4800,  0.4900,  0.3200,
          0.4900,  0.5700,  0.5500],
        [ 0.4800,  0.4900,  0.4700,  0.7400,  0.9700,  1.1000,  0.4700,
          0.4900,  0.5700,  0.5500]]) tensor([ 1,  0,  2])
torch.Size([3, 10]) torch.Size([3])
tensor([[ 0.4800,  0.4900,  0.3200,  0.7400,  0.9700,  1.1000,  0.3200,
          0.4900,  0.5700,  0.5500],
        [ 0.7400,  0.9700,  1.1000,  0.3200,  0.4800,  0.4900,  0.3200,
          0.4900,  0.5700,  0.5500],
        [ 0.4800,  0.4900,  0.4700,  0.7400,  0.9700,  1.1000,  0.4700,
          0.4900,  0.5700,  0.5500]]) tensor([ 1,  0,  2])


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f1ae21704e0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File

## In Torch

In [3]:
net = nn.Sequential(
    nn.Linear(dataset.x_dim[0], 50),
    nn.ReLU(),
    nn.Linear(50, dataset.y_dim[0]),
    nn.LogSoftmax()
)
############ NOT WORKING
print (net)
opt=optim.Adam(net.parameters())
print('torch.__version__',torch.__version__)
for epoch in range(2):
    print (epoch)
    for xb,yb in torch_dataloader:
        #print (type(xb),type(yb))
        if torch.__version__=='0.3.1': #starting in 0.4.0, it is done automagically
          xb = V(xb)
          yb = V(yb)
          
        pred = net(xb)
        #print ('pred',pred.shape,'yb',yb.shape)
        loss = nn.NLLLoss()(pred,yb)

        loss.backward()
        opt.step()
        opt.zero_grad()


Sequential(
  (0): Linear(in_features=10, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=3, bias=True)
  (3): LogSoftmax()
)
torch.__version__ 0.4.0
0
1


## In FastAI (shorter)

In [4]:
net = nn.Sequential(
    nn.Linear(dataset.x_dim[0], 50),
    #nn.ReLU(),
    nn.ELU(), #ELU is 2018 LeakyRelu is 2017 Relu is 2014
    nn.Linear(50, dataset.y_dim[0]),
    nn.LogSoftmax()
)


loss= nn.NLLLoss()
metrics=[accuracy]
opt=optim.Adam(net.parameters())

fastai_dataloader = fastai.dataloader.DataLoader(TimeDataset(1e3),batch_size=5)
md = ModelData('.', fastai_dataloader,fastai_dataloader)


fit(net, md, n_epochs=5, crit=loss, opt=opt, metrics=metrics)

# how to predict/eval on one-batch the result
x,y=next(iter(md.trn_dl))
print ('batch x',x.shape,'y',y.shape)
p = net(x) 
print ('p',p.max(dim=1)[1]) #max is pyTorch max,argmax together max is [0]
# we can do it in numpy toom but you an't call numpy() on Variable that requires grad.
print('p',np.argmax(p.detach().numpy(),axis=1))  

print ('y',y.numpy())  #here you can print y directly



epoch      trn_loss   val_loss   accuracy   
    0      0.642369   0.538236   1.0       
    1      0.452212   0.430325   1.0       
    2      0.3695     0.376724   1.0       
    3      0.300625   0.276684   1.0       
    4      0.205823   0.185157   1.0       

batch x torch.Size([5, 10]) y torch.Size([5])
p tensor([ 0,  0,  1,  2,  0])
p [0 0 1 2 0]
y [0 0 1 2 0]


## Let's implement another Time





In [107]:
from torch.utils.data import Dataset#, DataLoader
import numpy as np
import time

class TimePairsDataset(BaseDataset):
  
  def __init__(self,length,time_mod_3_result): #1e9 means 1970-2001
    """
    time_mod_3_result - to make sure time is train/test/valid is different, pass
    a different result here.
    """
    #super().__init__()
    self.length = length
    self.formats = ["%b %d %Y %H:%M:%S","%d %b %Y %H:%M:%S", "%m/%b/%Y %H:%M:%S"]
    self.time_mod_3_result=time_mod_3_result
  
  
  def __len__(self):
    return int(self.length)
  
  def _choose(self,idx,style):
    st = time.strftime(self.formats[style],time.gmtime(idx))
    return np.array([ord(c) for c in list(st)][:24],np.float32)/120 #chars are up to that
  
  def untokenize(self,sample):
    x,y = sample
    out=[] 
    for tokens in x:    
      out.append(''.join([chr(int(round(token*120))) for token in tokens]))
    return out,y

    
  
  
  def _tvt(self,idx):
    # for 10, if train, return 9, valid 10, test 11
    return (idx//3) * 3 + self.time_mod_3_result #10//3 * 3 + x = 9+x
  
  def __getitem__(self,idx):
    """ x -  x[0] semantic0   , style0
             x[1] semantic0   , style1   
             x[2] semantic0orX, style0 (1/2 the time 0 , half X)
    """
    
    #other_idx is 50% same, 50% other
    other_idx = (idx + np.random.randint(0,2)*(np.random.randint(10,self.length))) %self.length
    #print (idx,other_idx)
    idx = self._tvt(idx)
    other_idx = self._tvt(other_idx)
    #print (idx,other_idx)
    
    # two random formats
    random_fs= np.random.choice(len(self.formats),size=2,replace=False)
    #print (random_fs)
    x_list = []
    x_list.append(self._choose(idx,random_fs[0]))
    x_list.append(self._choose(idx,random_fs[1]))
    x_list.append(self._choose(other_idx,random_fs[0]))
    
    y = np.array(idx==other_idx,np.int64)
   
    return  (np.vstack(x_list), y)

def test():
  dataset = TimePairsDataset(1e7,1)
  sample = dataset[9] ;#print (sample)
  print (dataset.untokenize(sample))
  
  
test()  

(['Jan 01 1970 00:00:10', '01/Jan/1970 00:00:10', 'Jan 01 1970 00:00:10'], array(1))


In [115]:

import torch.nn as nn
import fastai
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *
from fastai.core import *
from fastai.imports import *
from fastai.torch_imports import *
from fastai.io import *
%matplotlib inline




# semantic_encoder : take sentence return a vector with only semantics
# semantic similiary objective: given a pair of sentences with the same semantics,
#    expect their results to be close (MeanSquared)    

# see idea from https://github.com/edenton/drnet-py/blob/master/models/classifiers.py
# in her train method:
# FRAMES: VID A: FRAME 1 x_c1 , ?x_p2? (1/2 of times)
#                FRAME 2 x_c2 , x_p1
#         VID B: FRAME 1      , ?x_p2? (1/2 of times)
#  
# x_p1,x_p2 chosen randonly from same/not-same video
# x_c1,x_c2 are always same-video
# c1 and p1 are same content, different pose.

# train_scene_discriminator()
# h_p1,h_p2 = netEP applied to x[0],x[1]. assumption: SAME VIDEO
# important: detach both!
# override half of h_p2 by random-permutations of the batch. 
#   [1,2,3,4,5,6]
#   [2,3,1,4,5,6] after 1st half permute
#   [1,1,1,0,0,0] set unequal the labels (1=unequal, 0 equal. does it matter? should it be 0.9,0.1?)
# apply BCE on inpit: concat of [h_p1,h_p2] 
# run backward, and optimizer on the netC classifier ONLY! emphasize! not on the encoder


# train()
# h_c1,h_c2 = netEC(x_c1), netEC(x_c2)  where input is x[0],x[1] sim loss is MSE directly on the hidden content-semantics
# h_p1,h_p2 = netEP(x_p1),netEP(x_p2) where input is x[2],x[3]
# rec = netD([h_c1, h_p1]) h_c1 is DIFFERENT FRAME , but same content, than h_p1
# netC is the semantic-discriminator given h_p1,h_p2, target 0.5 (max-entropy). 
# emphasize! don't optimize netC is this stage

 
md = ModelData('.',fastai.dataloader.DataLoader(TimePairsDataset(1e5,1), batch_size=16,),
                   fastai.dataloader.DataLoader(TimePairsDataset(1e4,2), batch_size=16,))
x,y = next(iter(md.trn_dl)) 
print ('input . x is vertically stacked sentences',x.shape,'y',y.shape)

batch,pair,sentence_len=x.shape

semantics_units = 20
style_units = 10


      
en_sem = nn.Sequential(
  # expect sentence_len,so from outside, cut a pair into x[0],x[1] and pass seperatly each
  nn.Linear(sentence_len,19),
  nn.ReLU(inplace=True),
  nn.Linear(19,semantics_units),
  nn.Tanh()
)

en_sty= nn.Sequential(
  nn.Linear(sentence_len,19),
  nn.ReLU(inplace=True),
  nn.Linear(19,style_units),
  nn.Tanh()
)
decoder = nn.Sequential(
  # input concat of semantic and style
  # output sentence_len , each word/char has currently value in range 0..1
  nn.Linear(semantics_units+style_units,19),
  nn.ReLU(inplace=True),
  nn.Linear(19,sentence_len),
  # we apply MSE on this
)
adv_dis = nn.Sequential(
  # input concat of two style
  nn.Linear(2*style_units,19),
  nn.ReLU(inplace=True),
  nn.Linear(19,1),
  nn.Sigmoid()  #Must be sigmoid, we apply later BCE
)


# trick: fastai requires model and loss function, but we have many small models together
# so we just wrap it up with big network, combine losses, and pass to fastai a lamba x:x loss
class MainNet(nn.Module):
  def __init__(self,en_sem,en_sty,decoder,adv_discriminator,verbose=False):
    super().__init__()
    self.en_sem = en_sem
    self.en_sty = en_sty
    self.decoder = decoder
    self.adv_discriminator = adv_discriminator
    self.verbose = verbose
    
  def forward(self,x): 
    if self.verbose: print ('\nMainNet x',x.numpy().shape,x[:,0].numpy().shape)
    
    # x[0] semantic0   , style0
    # x[1] semantic0   , style1   
    # x[2] semantic0orX, style0 (1/2 the time 0 , half X)
    
    sent0 = x[:,0]
    sent1 = x[:,1]
    sentX = x[:,2]
    h_sem0 = self.en_sem(sent0) 
    h_sem1 = self.en_sem(sent1) 
    # if you want to use torch criterion, you need to copy the label and set it to not requreing gradiant
    # so below is different than nn.MSELoss()(h_sem0,h_sem1.detach()). I wonder if only one get grad updates!
    loss_s = torch.mean(torch.pow(  h_sem0- h_sem1,2)) 
    h_sty0 = self.en_sty(sent0)
    recon_sent0 = self.decoder(torch.cat([h_sem1,h_sty0],dim=1)) #assume h_sem0==h_sem1
    recon_loss = nn.MSELoss()(recon_sent0,sent0)
    # TODO : willl it be better to use completely different sentences?
    h_styX = self.en_sty(sentX)
    adv_disc_p =  self.adv_discriminator(torch.cat([h_sty0,h_styX],dim=1))
    adv_target = np.full(shape =(sent0.shape[0],1),fill_value=0.5)
    adv_disc_loss = nn.BCELoss()(adv_disc_p, T(adv_target))
               
    self.verbose = False
    return loss_s + recon_loss + adv_disc_loss
 

 

#main_net = MainNet(en_sem, en_sty,decoder, adv_discriminator, verbose=True) 
#opt = optim.Adam(main_net.parameters())
#fit(main_net, md, n_epochs=10, crit=lambda loss,label:loss, opt=opt, metrics=[])



input . x is vertically stacked sentences torch.Size([16, 3, 20]) y torch.Size([16])


  0%|          | 0/6250 [00:00<?, ?it/s]
MainNet x (16, 3, 20) (16, 20)
(16, 20)
epoch      trn_loss   val_loss   
    0      0.693245   0.693267  
 82%|████████▏ | 5101/6250 [00:28<00:06, 176.04it/s, loss=0.693]

    1      0.693207   0.693232  
    2      0.6932     0.693232  
 49%|████▊     | 3041/6250 [00:17<00:18, 175.23it/s, loss=0.693]

KeyboardInterrupt: ignored

In [6]:
import argparse
import sys
parser = argparse.ArgumentParser()
parser.add_argument('--lr', default=0.002, type=float, help='learning rate')
parser.add_argument('--beta1', default=0.5, type=float, help='momentum term for adam')
parser.add_argument('--batch_size', default=100, type=int, help='batch size')
parser.add_argument('--log_dir', default='logs', help='base directory to save logs')
parser.add_argument('--data_root', default='', help='root directory for data')
parser.add_argument('--optimizer', default='adam', help='optimizer to train with')
parser.add_argument('--epocs', type=int, default=2, help='number of epochs to train for')
parser.add_argument('--seed', default=1, type=int, help='manual seed')
parser.add_argument('--epoch_size', type=int, default=600, help='epoch size')
parser.add_argument('--content_dim', type=int, default=128, help='size of the content vector')
parser.add_argument('--pose_dim', type=int, default=10, help='size of the pose vector')
parser.add_argument('--image_width', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--channels', default=3, type=int)
parser.add_argument('--dataset', default='moving_mnist', help='dataset to train with')
parser.add_argument('--max_step', type=int, default=20, help='maximum distance between frames')
parser.add_argument('--sd_weight', type=float, default=0.0001, help='weight on adversarial loss')
parser.add_argument('--sd_nf', type=int, default=100, help='number of layers')
parser.add_argument('--content_model', default='dcgan_unet', help='model type (dcgan | dcgan_unet | vgg_unet)')
parser.add_argument('--pose_model', default='dcgan', help='model type (dcgan | unet | resnet)')
parser.add_argument('--data_threads', type=int, default=5, help='number of parallel data loading threads')
parser.add_argument('--data_type', default='drnet', help='speed up data loading for drnet training')

sys.argv=["nothing"]
opt = parser.parse_args()


20


In [0]:

optimizer_en_sem = opt.optimizer(en_sem.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizer_en_sty = opt.optimizer(en_sty.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizer_decoder = opt.optimizer(decoder.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizer_adv_dis = opt.optimizer(adv_dis.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))


# --------- training funtions ------------------------------------
def train(x,nets,optimizers,nets):
    
    en_sty.zero_grad() 
    en_sem.zero_grad()
    decoder.zero_grad()

    
    # x[0] semantic0   , style0
    # x[1] semantic0   , style1   
    # x[2] semantic0orX, style0 (1/2 the time 0 , half X)
    sent0 = x[:,0]
    sent1 = x[:,1]
    sentX = x[:,2]
    h_sem0 = self.en_sem(sent0) 
    h_sem1 = self.en_sem(sent1) 
    # if you want to use torch criterion, you need to copy the label and set it to not requreing gradiant
    # so below is different than nn.MSELoss()(h_sem0,h_sem1.detach()). I wonder if only one get grad updates!
    sim_loss = torch.mean(torch.pow(  h_sem0- h_sem1,2)) 
    #sim_loss = mse_criterion(h_c1[0] if opt.content_model[-4:] == 'unet' else h_c1, h_c2)
    
    
    h_sty0 = self.en_sty(sent0)
    recon_sent0 = self.decoder(torch.cat([h_sem1,h_sty0],dim=1)) #assume h_sem0==h_sem1
    rec_loss = nn.MSELoss()(recon_sent0,sent0)

    # TODO : willl it be better to use completely different sentences?
    h_styX = self.en_sty(sentX)
    adv_disc_p =  self.adv_discriminator(torch.cat([h_sty0,h_styX],dim=1))
    adv_target = np.full(shape =(sent0.shape[0],1),fill_value=0.5)
    adv_disc_loss = nn.BCELoss()(adv_disc_p, T(adv_target))
    
    
    # full loss
    loss = sim_loss + rec_loss + opt.sd_weight*adv_disc_loss
    loss.backward()

    optimizer_en_sem.step()
    optimizer_en_sty.step()
    optimizer_decoder.step()

    return sim_loss.data.cpu().numpy(),rec_loss.data.cpu().numpy() 

def train_scene_discriminator(x,y):
    adv_disc.zero_grad()

    target = torch.cuda.FloatTensor(opt.batch_size, 1)

    h_sty0    = self.en_sty(x[0]) #detach?
    h_sty0or2 = self.en_sty(x[2])
    
  
    out = adv_discriminator([h_sty0, h_sty0or2])
    bce = bce_criterion(out, y) #should wrapp in varaible?

    bce.backward()
    optimizer_adv_dis.step()

    acc =  np.argmax(out.detach().numpy(),axis=1)==y  #CHECK THIS DIMENSTIONS!!! 
    return bce.data.cpu().numpy(), acc.data.cpu().numpy()/len(y)

# --------- training loop ------------------------------------
for epoch in rangeepocs):
    en_sty.train()
    en_sem.train()
    decoder.train()
    adv_discriminator.train()
    epoch_sim_loss, epoch_rec_loss, epoch_sd_loss, epoch_sd_acc = 0, 0, 0, 0

    for i in range(opt.epoch_size):
        print (i)
        x = next(training_batch_generator)

        # train scene discriminator
        sd_loss, sd_acc = train_scene_discriminator(x)
        epoch_sd_loss += sd_loss
        epoch_sd_acc += sd_acc

        # train main model
        sim_loss, rec_loss = train(x)
        epoch_sim_loss += sim_loss
        epoch_rec_loss += rec_loss


# Same for MNIST

In [38]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.io import *
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *

import torch.nn as nn


path = 'data/mnist/'
import os
os.makedirs(path, exist_ok=True)
URL='http://deeplearning.net/data/mnist/'
FILENAME='mnist.pkl.gz'

def load_mnist(filename):
    return pickle.load(gzip.open(filename, 'rb'), encoding='latin-1')
get_data(URL+FILENAME, path+FILENAME)
((x, y), (x_valid, y_valid), _) = load_mnist(path+FILENAME)

type(x), x.shape, type(y), y.shape
mean = x.mean()
std = x.std()

x=(x-mean)/std
mean, std, x.mean(), x.std()
x_valid = (x_valid-mean)/std
x_valid.mean(), x_valid.std()
print (x.shape,y.shape,x_valid.shape,y_valid.shape)
net = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ReLU(),
    nn.LogSoftmax()
)

md = ImageClassifierData.from_arrays(path, (x,y), (x_valid, y_valid))
loss=nn.NLLLoss()
metrics=[accuracy]
# opt=optim.SGD(net.parameters(), 1e-1, momentum=0.9)
opt=optim.SGD(net.parameters(), 1e-1, momentum=0.9, weight_decay=1e-3)
fit(net, md, n_epochs=1, crit=loss, opt=opt, metrics=metrics)

mnist.pkl.gz: 16.2MB [00:03, 4.75MB/s]                            


(50000, 784) (50000,) (10000, 784) (10000,)


epoch      trn_loss   val_loss   accuracy   
    0      0.96713    0.883354   0.8699    



[0.8833535579681396, 0.8699]